# Analisis de comentarios de peliculas

## 1. Importacion de librerias

In [1]:
import pandas as pd
import numpy as np
import re

# Librerias para el preprocesamiento
import spacy
nlp = spacy.load('es_core_news_sm')

# Librerias para transformar los datos
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin



# Librerias para vectorizar 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Librerias para el modelo
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

# Librerias para la evaluacion
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

# Librerias para la busqueda de hiperparametros
from sklearn.model_selection import GridSearchCV

# Librerias para el pipeline
from sklearn.pipeline import Pipeline

# Librerias para exportar el modelo
from joblib import dump, load



## 2. Carga de datos

In [2]:
# Carga del csv entrenamiento
file_name = './data/MovieReviews.csv'
raw = pd.read_csv(file_name, sep=',')
reviews = raw.copy()

In [3]:
# Carga del csv prueba
file_name = './data/MovieReviewsPruebas.csv'
raw = pd.read_csv(file_name, sep=',')
reviews_test = raw.copy()

## 3. Entendimiento de los datos

In [4]:
reviews.shape

(5000, 3)

In [5]:
# Tipo de variavles
reviews.dtypes

Unnamed: 0      int64
review_es      object
sentimiento    object
dtype: object

In [6]:
# Ver aproximadamente la cantidad de datos por la columna sentimiento
reviews['sentimiento'].value_counts()

positivo    2500
negativo    2500
Name: sentimiento, dtype: int64

In [7]:
reviews.head(3)

,Unnamed: 0,review_es,sentimiento
0,0,Si está buscando una película de guerra típica...,positivo
1,1,Supongo que algunos directores de películas de...,positivo
2,2,Es difícil contarle más sobre esta película si...,positivo


## 4. Creacion de los pipelines

### Transformadores

In [8]:
# Funcion para limpiar el texto utilizando spacy
def clean_text(text):
    # Pasar a minusculas
    text = text.lower()

    # Eliminar las tildes
    text = re.sub(r'[á]', 'a', text)
    text = re.sub(r'[é]', 'e', text)
    text = re.sub(r'[í]', 'i', text)
    text = re.sub(r'[ó]', 'o', text)
    text = re.sub(r'[ú]', 'u', text)
    
    # Eliminar los numeros
    text = re.sub(r'\d+', ' ', text)

    # Eliminar los signos de puntuacion
    text = re.sub(r'[^\w\s]', ' ', text)

    # Tokenizar
    tokens = nlp(text)

    # Eliminar stopwords
    tokens = [token.text for token in tokens if not token.is_stop]

    # Unir los tokens
    text = ' '.join(tokens)

    return text

In [9]:
# Creacion de un transformador para el preprocesamiento
class CleanTransformer (BaseEstimator, TransformerMixin):
    
    # Constructor
    def __init__(self, column):
        print('\n Inicializando el transformador CleanTransformer')
        self.column = column

    # Fit
    def fit(self, X, y=None):
        print('\n Fit del transformador CleanTransformer')
        return self
    
    # Transform
    def transform(self, X, y=None):
        print('\n Transform del transformador CleanTransformer')
        X_ = X.copy()
        X_[self.column] = X_[self.column].apply(clean_text)
        print(X_)
        return X_
        

### Pipeline

In [10]:
pipe = Pipeline([
                ('tfidf', TfidfVectorizer(preprocessor=clean_text, max_features=1800)),
                ('model', MultinomialNB(alpha=1.0))
])

## 5. Ejecucion y análisis

### Entrenamiento del modelo

In [11]:
# Separacion de las variables de decision y de prediccion
X = reviews['review_es']
Y = reviews['sentimiento']

In [12]:
# Creacion de los conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
y_train = y_train.values.ravel()

In [13]:
# Entrenamiento del modelo
model = pipe.fit(X_train, y_train)

### Analisis de metricas

In [14]:
# Realizar predicciones en el conjunto de entrenaiento y prueba
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [15]:
# Calcular las metricas de evaluacion
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, pos_label='positivo')
train_recall = recall_score(y_train, y_train_pred, pos_label='positivo')
train_f1 = f1_score(y_train, y_train_pred, pos_label='positivo')

test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, pos_label='positivo')
test_recall = recall_score(y_test, y_test_pred, pos_label='positivo')
test_f1 = f1_score(y_test, y_test_pred, pos_label='positivo')

In [16]:
# Imprimir las metricas de evaluacion
print('\nMetricas del conjunto de entrenamiento:')
print("Accuracy:",  train_accuracy)
print("Precision:", train_precision)
print("Recall:",    train_recall)
print("F1 score:",  train_f1)

# Imprimir las metricas de evaluacion
print('\nMetricas del conjunto de prueba:')
print("Accuracy:",  test_accuracy)
print("Precision:", test_precision)
print("Recall:",    test_recall)
print("F1 score:",  test_f1)


Metricas del conjunto de entrenamiento:
Accuracy: 0.869
Precision: 0.8592023065833734
Recall: 0.8855869242199108
F1 score: 0.8721951219512195

Metricas del conjunto de prueba:
Accuracy: 0.81
Precision: 0.7963340122199593
Recall: 0.8128898128898129
F1 score: 0.8045267489711934


### Busqueda de hiperparametros

Quizas con un poder computacional mucho mayor se pueda hacer la busqueda de hiperparametros

In [17]:
'''
# Definir los valores para los hiperparametros
param_grid = {
    'tfidf__max_features': [100, 500, 1000],
    'model__alpha': [1.0],
}

# Crear un objeto GridSearchCV
grid_search = GridSearchCV(pipe, param_grid, cv=5)

# Ajustar el modelo con los hiperparametros
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparametros
best_params = grid_search.best_params_
print('Valores óptimos de los hiperparámetros:', best_params)
'''

"\n# Definir los valores para los hiperparametros\nparam_grid = {\n    'tfidf__max_features': [100, 500, 1000],\n    'model__alpha': [1.0],\n}\n\n# Crear un objeto GridSearchCV\ngrid_search = GridSearchCV(pipe, param_grid, cv=5)\n\n# Ajustar el modelo con los hiperparametros\ngrid_search.fit(X_train, y_train)\n\n# Obtener los mejores hiperparametros\nbest_params = grid_search.best_params_\nprint('Valores óptimos de los hiperparámetros:', best_params)\n"

## 6. Exportar e importar el modelo

In [18]:
# Nombre del archivo donde se guardara el modelo
pipe_file_name = 'pipelineTFIDF.joblib'

# Guardar el modelo
dump(model, pipe_file_name)

['pipelineTFIDF.joblib']

In [19]:
# Cargar el modelo
test_model = load(pipe_file_name)
test_model

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=1800,
                                 preprocessor=<function clean_text at 0x12ffbf880>)),
                ('model', MultinomialNB())])

In [20]:
# Clasificar los datos de prueba
reviews_test['sentimiento'] = test_model.predict(reviews_test['review_es'])

In [21]:
# Visualizar los resultados
reviews_test

,Unnamed: 0,review_es,sentimiento
0,0,La saga medieval alemana de Fritz Lang continú...,positivo
1,1,Este anime es una visita obligada para los fan...,positivo
2,2,Esta es una de las mejores películas para masc...,positivo
3,3,"Cuando se lanzó su DVD, llegué al mercado y lo...",positivo
4,4,No me he reído tan duro en una película en muc...,positivo
...,...,...,...
295,295,1er vimos 2/18/2007 - 4 de 10 (Dir-Leon Leonar...,negativo
296,296,Desde su título no inspirado a las actuaciones...,negativo
297,297,Encontré a esta buena película para pasar su t...,positivo
298,298,Solía ​​trabajar en la compañía que originalme...,negativo
